In [1]:
import sys
sys.path.append("..")
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
from scipy import optimize
from writeit import rewritePipes


Populating the interactive namespace from numpy and matplotlib


In [2]:
def MyBounds(**kwargs):
        x = kwargs["x_new"]
        tmin = bool(np.all(x>=0))
        tmax = bool(np.all(x<=200.))
        print x
        print tmin
        print tmax
        return (tmax and tmin)

In [3]:
def print_fun(x, f, accepted):
      print("at minima %.4f accepted %d" % (f, int(accepted)))

In [4]:
D = 1.            #pipe diameter (m)
Mr = 0.015        #manning roughness coeffs
T = 20            #simulation time (s)
M = 320           #number of time steps
a = 10            #pressure wavespeed
Q00 = 1.#boundary condition at node 0

fn = "../indata/optlength"
oldinp = "../indata/3pipesL0.inp"
params= (D,Mr,T,M,a,Q00,fn,oldinp)


In [5]:
def f(z,*params):
    l1,l2,l3 = z
    D,Mr,T,M,a,Q00,fn,oldinp = params
    dt = T/float(M)
    
    Ns = [int(zi*.8/(dt*a)) for zi in z]
    Ls = [l1,l2,l3]
    Mrs = [Mr]*3  #manning roughness coeffs
    Ds = [D]*3      #pipe diameter (m)
    elevs = [0,0,0,0];  #junction elevations
    h0s = [.8,.8,.8]    #IC for h
    q0s = [2.,1.,1.]    #IC for Q
    jt = [1,3,1,1]   #junction type
    bt = [1,1,1,1]   #boundary type 
    bv = [0,0,0,0]   #boundary value
    r = [0,0,1,1]    #boundary reflect
    
    (fi, fc) = rewritePipes(fn,oldinp, Ns, Ls, Mrs, Ds, jt, bt, bv, r, h0s, q0s, T, M, a, elevs)
    n0 = PyNetwork(fi, fc, 1)
    n0.setbVal(0,Q00*np.ones(n0.M+1))

    n0.runForwardProblem(dt)
    H = mean([n0.getAveGradH(i) for i in range(n0.M+1)])
    print "L = [%f %f %f] and H = %f"%(l1,l2,l3,H)
    return H

In [ ]:
%%time
x0 = [150,50,100]
minimizer_kwargs = {"method": "BFGS", "args":params}
ret = optimize.basinhopping(f, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=200,accept_test=MyBounds, callback=print_fun)

new files are ../indata/optlength.inp and ../indata/optlength.config
L = [150.000000 50.000000 100.000000] and H = 1.791535
new files are ../indata/optlength.inp and ../indata/optlength.config
L = [150.000000 50.000000 100.000000] and H = 1.791535
new files are ../indata/optlength.inp and ../indata/optlength.config
L = [150.000000 50.000000 100.000000] and H = 1.791535
new files are ../indata/optlength.inp and ../indata/optlength.config
L = [150.000000 50.000000 100.000000] and H = 1.791535
new files are ../indata/optlength.inp and ../indata/optlength.config
L = [150.000000 50.000000 100.000000] and H = 1.791535
new files are ../indata/optlength.inp and ../indata/optlength.config
L = [149.000000 50.000000 100.000000] and H = 1.791191
new files are ../indata/optlength.inp and ../indata/optlength.config
L = [149.000000 50.000000 100.000000] and H = 1.791191
new files are ../indata/optlength.inp and ../indata/optlength.config
L = [149.000000 50.000000 100.000000] and H = 1.791191
new file

In [ ]:
res